In [10]:
import pandas as pd
import numpy as np
data=pd.read_csv("origindata.csv")
print(data.index)
%matplot inline

RangeIndex(start=0, stop=1048575, step=1)


UsageError: Line magic function `%matplot` not found.


In [9]:
#Index(['flight', 'name', 'date', 'time'], dtype='object')
name_index=set(data["name"].values)
date_index=set(data["date"].values)
flight_index=set(data["flight"].values)
merged_data={}
merged_data["flight"]=[]
merged_data["date"]=[]
for i in name_index:
    merged_data[i]=[]
merged_data

{360.0: [],
 361.0: [],
 364.0: [],
 365.0: [],
 366.0: [],
 367.0: [],
 368.0: [],
 369.0: [],
 370.0: [],
 378.0: [],
 379.0: [],
 380.0: [],
 637.0: [],
 641.0: [],
 644.0: [],
 645.0: [],
 648.0: [],
 649.0: [],
 652.0: [],
 654.0: [],
 655.0: [],
 656.0: [],
 660.0: [],
 661.0: [],
 662.0: [],
 663.0: [],
 664.0: [],
 665.0: [],
 668.0: [],
 669.0: [],
 670.0: [],
 675.0: [],
 676.0: [],
 678.0: [],
 679.0: [],
 680.0: [],
 681.0: [],
 682.0: [],
 683.0: [],
 684.0: [],
 685.0: [],
 'date': [],
 'flight': []}

In [12]:
#处理数据，将航班和字段提取出来得到各个保障节点的数据
for date in date_index:
    data_temp=data[data["date"]==date]
    flight_set=set(data_temp["flight"].values)
    for flight_temp in flight_set:
        merged_data["date"].append(date) #增加航班日期
        merged_data["flight"].append(flight_temp) #增加航班号
        for node_name in name_index:
            
            
    

        flight   name       date      time
136402  CZ6851  641.0  2017/6/29    9:45:0
207774  Y87935  367.0  2017/6/29    2:30:0
209266  Y87991  367.0  2017/6/29    3:20:0
210763   CZ457  367.0  2017/6/29    2:36:0
211282  CZ8705  367.0  2017/6/29   10:10:0
218381   ET607  367.0  2017/6/29    1:13:0
219080  Y87979  367.0  2017/6/29     6:9:0
219113  MU5734  367.0  2017/6/29    5:58:0
219114  FM9307  645.0  2017/6/29    4:41:0
219115  ZH9824  364.0  2017/6/29    4:41:0
219116  CA4320  366.0  2017/6/29   4:41:46
219117  ZH9973  370.0  2017/6/29    4:42:0
219118  CA4320  669.0  2017/6/29   4:42:20
219119  CA1316  637.0  2017/6/29   4:42:45
219120  CZ3707  644.0  2017/6/29    4:40:0
219121  CZ6980  662.0  2017/6/29    4:42:0
219122   CZ339  369.0  2017/6/29    4:42:0
219123  CZ6980  660.0  2017/6/29    4:42:0
219124  CA8232  661.0  2017/6/29    4:42:0
219125  CZ3109  669.0  2017/6/29    4:42:0
219126  MU5744  670.0  2017/6/29    4:43:0
219127  CZ3204  364.0  2017/6/29    4:43:0
219128  CA8